# Regression. Part 1
---
Author: Anatoliy Durkin

Updated: 24.03.2025

---
В данном ноутбуке будут рассмотрены регрессионные модели, метрики численного прогнозирования и методы подготовки признаков.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Линейная регрессия

In [ ]:
df = pd.read_csv('linear_regression.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['x']], df['y'], test_size=0.2, random_state=42)

In [ ]:
df.plot.scatter(x='x', y='y', xlabel='x', ylabel='y', title='Data visualisation', grid=True)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
reg.coef_

In [ ]:
reg.intercept_

In [ ]:
df.plot.scatter(x='x', y='y', xlabel='x', ylabel='y', title='Linear regression', grid=True)
plt.plot([0, 100], [0*reg.coef_[0]+reg.intercept_, 100*reg.coef_[0]+reg.intercept_], color='red', linewidth=3)

Линейная регрессия прекрасно ложится на наши данные. Посмотрим, какую оценку модели мы получаем.

In [ ]:
print('Train:', reg.score(X_train, y_train))
print('Test:', reg.score(X_test, y_test))

Однако, что это за метрика? Чаще всего в процессе обучения модели используют квадрат ошибки в функциях потерь. Возможно, эта метрика нам и показана.

MSE - mean squared error.

$$MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i-f(x_i))^2$$

Из-за возведения в квадрат не очень понятно, как интерпретировать эту метрику, поэтому зачастую берут корень из этой метрики: RMSE - root mean squared error.

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
print('MSE:', MSE(reg.predict(X_test), y_test))
print('RMSE:', MSE(reg.predict(X_test), y_test)**0.5)

Метрики не похожи по значению на полученные ранее.

Поскольку MSE и RMSE никак не ограничены сверху, иногда их очень сложно интерпретировать и понимать, хорошее ли это значение. Поэтому придумана метрика, изменяющаяся от 0 до 1 - $R^2$ или коэффициент детерминации. Этот показатель отражает, какая доля вариативности зависимой переменной объясняется независимыми переменными в модели.

$$R^2 = 1 - \frac{\sum_{i=1}^{N} (y_i-f(x_i))^2}{\sum_{i=1}^{N} (y_i-\bar y)^2}$$

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print('R2:', r2_score(reg.predict(X_test), y_test))

Теперь мы получили ту самую метрику. Чаще всего именно $R^2$ является очновной метрикой для регрессионных моделей. Чем ближе к единице, тем лучше. 

Хотя R-квадрат изменяется от 0 до 1, на практие можно столкнуться с тем, что эта метрика окажется отрицательной. Это свидельствует о том, что построенная модель предсказывает хуже, чем константная модель.

Какие ещё метрики можно использовать при оценке модели?

Самое простое посмотреть, на сколько мы ошибаемся в абсолютных значениях.

MAE - mean absolute error. А когда мы говорим об относительных значениях, используем MAPE - mean absolute percentage error.

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE, mean_absolute_percentage_error as MAPE

In [ ]:
print('MAE:', MAE(reg.predict(X_test), y_test))
print('MAPE:', MAPE(reg.predict(X_test), y_test))

Какие ещё метрики вы бы могли использовать?

Например, доля предсказаний с абсолютными ошибками больше некоторого значения. Попробуйте посчитать такое значение.

In [ ]:
# Ваш код
...

# Кодирование признаков

In [ ]:
df = pd.read_csv('insurance.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
sns.pairplot(df)

In [ ]:
sns.heatmap(df.corr(), cmap='bwr', center=0, annot=True)

Что в данных помешает нам сразу же построить хоть какую-нибудь модель?

Строковый тип данных. Модели в основном не умеют работать с текстом, есть те, что умеют обрабатывать категориальные данные, как в нашем случае, но для остальных моделей, как и для линейной регрессии, данные должны быть численными.

У нас есть три столбца для обработки: `sex`, `smoker`, `region`. Проще всего начать со столбца `smoker`. Что представляют собой данные? Как их можно записать в числах?

In [ ]:
df['smoker'].unique()

По сути, это бинарный признак, принимающий два значения - "да" и "нет", которые также можно записать `True` и `False`, булево значение. А значит данные этого столбца можно смело заменить на нули и единицы.

In [ ]:
df['smoker'] = df['smoker'].apply(lambda x: 0 if x=='no' else 1)

In [ ]:
df['smoker'].unique()

Перейдём к столбцу `sex`, какие тут уникальные значения?

In [ ]:
# Ваш код
...

Всего два категориальных значения, но они не представимы сразу булевым типом. Что с ними можно сделать?

Да, их можно просто занумеровать нулями и единицами (главное запомнить, что есть что). Сделайте это.

In [ ]:
# Ваш код
...

И вот мы добрались до третьего столбца, `region`, какие значения встречаются в нем?

In [ ]:
# Ваш код
...

Теперь поговорим о двух методах превращения категориальных столбцов в численные - прямое кодирование и порядковое кодирование.

Техника прямого кодирования, или отображения (англ. One-Hot Encoding, OHE). Принцип действия:
- Для каждого значения признака создаётся новый столбец.
- Если объекту категория подходит, присваивается 1, если нет — 0.

для прямого кодирования в библиотеке `pandas` есть функция `get_dummies()`.

In [ ]:
pd.get_dummies(df['region']).head()

Когда данных много, можно угодить в ловушку фиктивных признаков, это не очень хорошо для моделей. Также она называется дамми-ловушка (англ. dummy trap, «ловушка фиктивных признаков»).

Посмотрите на полученные сверху четыре столбца. Один из них можно смело удалять, ведь его легко восстановить из оставшихся, он избыточен. Для этого у функции `get_dummies()` есть аргумент `drop_first`. Если указать `True`, то первый из столбцов будет удалён.

In [ ]:
pd.get_dummies(df['region'], drop_first=True).head()

Обратите внимание, в целом это очень похоже на то, что мы сделали с двумя другими столбцами. Если их обработать техникой прямого кодирования и удалить один из столбцов, получится то же самое. Собственно, функция `get_dummies()` умеет сама определять категориальные переменные и обрабатывать их. Попробуйте передать в функцию весь датасет и посмотрите, как обработаются интересующие нас столбцы.

In [ ]:
df = pd.read_csv('insurance.csv')

In [ ]:
# Ваш код
...

Второй подход - техника порядкового кодирования. Она подходит для порядковых категориальных признаков.

Есть функция. позволяющая закодировать цифрами выраженные в тексте категории —  Ordinal Encoding (от англ. «кодирование по номеру категории»). Она работает так:
- Фиксируется, какой цифрой кодируется класс.
- Цифры размещаются в столбце.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv('insurance.csv')

In [ ]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(df), columns=df.columns)

In [ ]:
data_ordinal.head()

Как выбрать одну из техникам кодирования категориальных переменных?
- Если все признаки должны стать количественными, подходит техника OHE.
- Когда все признаки категориальные, и их нужно преобразовать в числа — Ordinal Encoding.

Определитесь с наилучшей техникой для имеющихся данных и одготовьте набор данных для дальнейшей работы. Посмотрите также на парные распределения у получившихся столбцов и матрицу кореляций.

In [ ]:
df = pd.read_csv('insurance.csv')

In [ ]:
# Ваш код
...

# Множественная линейная регрессия

Линейная регрессия может строиться и на нескольких признаках, тогда получается некая прямая в многомерном пространстве, которую далеко не всегда можно визуализировать.

Разделите подготовленные данные на `features` и `target` и постройте модель `LinearRegression`.

In [ ]:
# Ваш код
...

In [ ]:
reg.score(features, target)

In [ ]:
reg.coef_

In [ ]:
reg.intercept_

Для того, чтобы увидеть больше разнообразных метрик, позволяющих оценить качество модели и подбирать набор признаков, используем аналогичную модель из другой библиотеки.

In [ ]:
import statsmodels.api as sm

В данном случае свободную переменную необходимо добавлять вручную, также можно не делать этого и строить регрессию без свободной переменной.

In [ ]:
features = sm.add_constant(features)

In [ ]:
model = sm.OLS(target, features)

In [ ]:
results = model.fit()

In [ ]:
results.params

Сравните коэффициенты двух моделей.

Но самым главным плюсом использования данной библиотеки является метод `summaries()`, представляющий подробную сводку об оценке модели.

In [ ]:
results.summary()

Вывод достаточно объёмный и состоит из трех частей.

- В первой представлены основные оценки модели, где присутсвует в том числе разобранный выше $R^2$.
- Во второй части дана информация по коэффициентам.
- Третья часть представляет анализ остатков.

Подробное описание можно прочитать [в статье](https://habr.com/ru/articles/681218/).

Нас сейчас интересует вторая часть, так как она позволяет производить отбор признаков.

В этой таблице присутствуют сами подобранные коэффициенты; дисперсия коэффициента `std err`; t-критерий Стьюдента; p-value, позволяющее принимать и отвергать гипотезы о значимости коэффициентов; доверительные интервалы коэффициентов.

Как раз по p-value можно смотреть, важен ли тот или иной признак для модели. Если p-value меньше 0.05 (или 0.01, два классических значения для оценки), то гипотеза о значимости принимается. В ином случае нулевая гипотеза отвергается, признак можно считать неважным. Значит, его можно удалять.

Важно! Если по таблице гипотеза отвергается сразу для нескольких признаков, не стоит удалять их все сразу - после удаления одного p-value для других могут значительно измениться. Поэтому убираем признаки по одному, начиная с самого большого p-value. Останавливаемся, когда p-value всех признаков будет укладываться в заданые рамки.

Попробуйте удалять признаки и заново строить регрессию. Станут ли лучше метрики?

In [ ]:
# Ваш код
...